In [2]:
import numpy as np
import matplotlib.pyplot as plt


In [3]:
from nnfs.datasets import spiral_data
import numpy as np
import nnfs
nnfs.init()
import matplotlib.pyplot as plt
X, y = spiral_data(samples=100, classes=3)

In [4]:
np.random.randn(4,3)
np.zeros((1,3))

array([[0., 0., 0.]], dtype=float32)

In [5]:
### import dataset
from nnfs.datasets import spiral_data
import nnfs
nnfs.init()
import numpy as np
import matplotlib.pyplot as plt
# X, y = spiral_data(samples=100, classes=3)

In [21]:
## coding the layer dence to create neruon layers
# Dense layer
class Layer_Dense:
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    # Forward pass
    def forward(self, inputs):
        # Remember input values
        self.inputs = inputs
        # Calculate output values from input ones, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass
    def backward(self, dvalues):
        # Gradients on parameters
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)




## relu acivation class
class activation_relu:
    def forward(self,input):
        self.output=np.maximum(0,input)
    
    def backward(self,dvalue):
        self.dinput=self.dvalue.copy
        self.dinput[self.dvalue<=0]=0
    

class activation_softmax:
    def forward(self,input):
        evalue=np.exp(input-np.max(input,axis=1,keepdims=True))
        probabilities=evalue/np.sum(evalue,axis=1,keepdims=True)
        self.output=probabilities
#define the loss class parent and child cross entropy loss

class Loss:
  
        
    def calculate(self,y_pred,y_true):
        output=self.forward(y_pred,y_true)
        return np.sum(output)
class Loss_CategoricalCrossentropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
                y_pred_clipped * y_true,
                axis=1
            )

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # We'll use the first sample to count them
        labels = len(dvalues[0])

        # If labels are sparse, turn them into one-hot vector
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate gradient
        self.dinputs = -y_true / dvalues
        # Normalize gradient
        self.dinputs = self.dinputs / samples

    






    
        

# Softmax classifier - combined Softmax activation
# and cross-entropy loss for faster backward step
class Activation_Softmax_Loss_CategoricalCrossentropy:
    # Creates activation and loss function objects
    def __init__(self):
        self.activation = activation_softmax()
        self.loss = Loss_CategoricalCrossentropy()

    # Forward pass
    def forward(self, inputs, y_true):
        # Output layer's activation function
        self.activation.forward(inputs)
        # Set the output
        self.output = self.activation.output
        # Calculate and return loss value
        return self.loss.calculate(self.output, y_true)

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # If labels are one-hot encoded,
        # turn them into discrete values
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        # Copy so we can safely modify
        self.dinputs = dvalues.copy()
        # Calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # Normalize gradient
        self.dinputs = self.dinputs / samples
            

    

        


<div class="alert alert-block alert-success">
<b>FULL CODE UPTO THIS POINT: FORWARD AND BACKWARD PASS</b>
</div>

In [7]:
# import dataset
x,y=spiral_data(samples=100,classes=3)
# initalize all the classes
dence1=Layer_Dense(2,3)
activation1=activation_relu()
dence2=Layer_Dense(3,3)
# activation2=activation_softmax()
loss_activatiion=Activation_Softmax_Loss_CategoricalCrossentropy()
back=Activation_Softmax_Loss_CategoricalCrossentropy()


# froward pass
dence1.forward(x)
activation1.forward(dence1.output)
dence2.forward(activation1.output)
loss=loss_activatiion.forward(dence2.output,y)



print('loss:', loss)


predictions = np.argmax(loss_activatiion, axis=1)
if len(y.shape) == 2:
 y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)
# Print accuracy
print('acc:', accuracy)




AttributeError: 'Activation_Softmax_Loss_CategoricalCrossentropy' object has no attribute 'forward'

In [ ]:
## optimizer SGD  with momentom


<div class="alert alert-block alert-success">
<b>OPTIMIZERS: MOMENTUM</b>
</div>

In [8]:
class optimizer_SGD:
    def __init(self,learning_rate=1.,decay=0.,momentum=0.):
        self.currenT_learning_rate=learning_rate
        self.learning_rate=learning_rate
        self.decay=decay
        self.momentum=momentum
        self.iteration=0


    ## call befor parameter updates
    def pre_updates_params(self):
        if  self.decay:
            self.currenT_learning_rate=self.learning_rate/(1+self.decay*self.iteration)

    ## update params
    def update_params(self,layer):
        if self.momentum:
                        # If layer does not contain momentum arrays, create them
            # filled with zeros
            # if not hasattr(layer, 'weight_momentums'):
            #     layer.weight_momentums = np.zeros_like(layer.weights)
            #     layer.bias_momentums = np.zeros_like(layer.biases)
            if not hasattr(layer,'previus_weights'):
                layer.previus_weights=np.zeros_like(layer.weights)
                layer.previus_biases=np.zeros_like(layer.biases)
            weight_update=(self.momentum*layer.previus_weights)-(self.currenT_learning_rate*layer.layer.dweights)
            layer.previus_weights=weight_update
            ## bieas
            biases_updates=(self.momentum*layer.previus_biases)-(self.currenT_learning_rate*layer.layer.dbiases)
            layer.previus_biases=biases_updates
        else:# vanila SGD
            weight_update=-(self.currenT_learning_rate*layer.layer.dweights)
            biases_updates=-(self.currenT_learning_rate*layer.layer.dbiases)
        
        layer.weights += weight_update
        layer.biases += biases_updates

    def post_updates(self):
        self.iteration+=1
        


        



In [15]:
import numpy as np

class optimizer_SGD_1:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__(self, learning_rate=1., decay=0., momentum=0.):
        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum

    # Call once before any parameter updates
    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))

    # Update parameters
    def update_params(self, layer):
        # If we use momentum
        if self.momentum:
            # If layer does not contain momentum arrays, create them
            # filled with zeros
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            # Build weight updates with momentum - take previous
            # updates multiplied by retain factor and update with
            # current gradients
            weight_updates = self.momentum * layer.weight_momentums - \
                             self.current_learning_rate * layer.dweights
            layer.weight_momentums = weight_updates

            # Build bias updates
            bias_updates = self.momentum * layer.bias_momentums - \
                           self.current_learning_rate * layer.dbiases
            layer.bias_momentums = bias_updates

        # Vanilla SGD updates (as before momentum update)
        else:
            weight_updates = -self.current_learning_rate * layer.dweights
            bias_updates = -self.current_learning_rate * layer.dbiases

        # Update weights and biases using either
        # vanilla or momentum updates
        layer.weights += weight_updates
        layer.biases += bias_updates

    # Call once after any parameter updates
    def post_update_params(self):
        self.iterations += 1


In [22]:
x,y=spiral_data(samples=100,classes=3)
dence1=Layer_Dense(2,64)
activation1=activation_relu()

dence2=Layer_Dense(64,3)
loss_activatiion_2=Activation_Softmax_Loss_CategoricalCrossentropy()
optimizer = optimizer_SGD_1(decay=1e-3, momentum=0.9)

for epoch in range(10000):
    # forward pass
    dence1.forward(x)
    activation1.forward(dence1.output)
    dence2.forward(activation1.output)
    loss=loss_activatiion_2.forward(dence2.output,y)

    ## accuaracy
    predictions=np.argmax(loss_activatiion_2.output,axis=1)
    #     if len(y.shape) == 2:
    #     y = np.argmax(y, axis=1)
    # accuracy = np.mean(predictions == y)
    if len(y.shape)==2:
        y=np.argmax(y,axis=1)
    accuracy=np.mean(predictions==y)

    if not epoch%100:
        print(f"epoch : {epoch}, accuracy : {accuracy:.3f} , loss:{loss:.3f}",'lr: {optimizer.current_learning_rate}' )

    #    # Backward pass
    # loss_activation.backward(loss_activation.output, y)
    # dense2.backward(loss_activation.dinputs)
    # activation1.backward(dense2.dinputs)
    # dense1.backward(activation1.dinputs)
    # backward pass
    loss_activatiion_2.backward(loss_activatiion_2.output,y)
    
    dence2.backward(loss_activatiion_2.dinputs)
    activation1.backward(dence2.dinputs)
    dence1.backward(activation1.dinput)

    ## updates weights and biases
    """"    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()"""
    optimizer.pre_updates_params()
    optimizer.update_params(dence1)
    optimizer.update_params(dence2)
    optimizer.post_updates()
    


    

   


epoch : 0, accuracy : 0.283 , loss:329.586 lr: {optimizer.current_learning_rate}


AttributeError: 'activation_relu' object has no attribute 'dvalue'

In [23]:
import numpy as np

# Assuming the necessary classes (Layer_Dense, Activation_ReLU, 
# Activation_Softmax_Loss_CategoricalCrossentropy, Optimizer_SGD, and spiral_data) are defined elsewhere

# Create dataset
X, y = spiral_data(samples=100, classes=3)

# Create Dense layer with 2 input features and 64 output values
dense1 = Layer_Dense(2, 64)

# Create ReLU activation (to be used with Dense layer)
activation1 = activation_relu()

# Create second Dense layer with 64 input features (as we take output
# of previous layer here) and 3 output values (output values)
dense2 = Layer_Dense(64, 3)

# Create Softmax classifier's combined loss and activation
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()

# Create optimizer
optimizer = optimizer_SGD_1(decay=1e-3, momentum=0.9)

# Train in loop
for epoch in range(10001):
    # Perform a forward pass of our training data through this layer
    dense1.forward(X)
    
    # Perform a forward pass through activation function
    # takes the output of first dense layer here
    activation1.forward(dense1.output)
    
    # Perform a forward pass through second Dense layer
    # takes outputs of activation function of first layer as inputs
    dense2.forward(activation1.output)
    
    # Perform a forward pass through the activation/loss function
    # takes the output of second dense layer here and returns loss
    loss = loss_activation.forward(dense2.output, y)
    
    # Calculate accuracy from output of activation2 and targets
    # calculate values along first axis
    predictions = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(predictions == y)
    
    if not epoch % 100:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy:.3f}, ' +
              f'loss: {loss:.3f}, ' +
              f'lr: {optimizer.current_learning_rate}')
    
    # Backward pass
    loss_activation.backward(loss_activation.output, y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)
    
    # Update weights and biases
    optimizer.pre_update_params()
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.post_update_params()


epoch: 0, acc: 0.363, loss: 329.575, lr: 1.0


AttributeError: 'activation_relu' object has no attribute 'dvalue'